In [3]:
from sklearn.decomposition import PCA
import hopsworks
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [4]:
project = hopsworks.login(api_key_value="wOWQmzzHeITT5wlJ.FkOButuQ3XpDXSUO1LnTuaNfD7SPWolfF1hateoistsLFFsBF7upULn5z6qKPOIB")
fs = project.get_feature_store() 
headline_fg = fs.get_feature_group(
    name='headlinesemb', 
    version=1,
)
headline_df = headline_fg.read()
headline_df = headline_df.sort_values(by='date')

print(headline_df.head())
print(headline_df.tail())

2024-12-30 10:18:41,010 INFO: Initializing external client
2024-12-30 10:18:41,012 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-30 10:18:42,294 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163414
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.28s) 
          dim_0     dim_1     dim_2     dim_3     dim_4     dim_5     dim_6  \
3016   0.009504 -0.000808  0.018312 -0.061341 -0.046319  0.025388 -0.024238   
10784  0.035778  0.061700  0.076458  0.039224  0.054754 -0.023615 -0.099850   
7168   0.043952  0.065020  0.051891  0.002397  0.041112 -0.036826  0.000049   
10864 -0.034407 -0.044771 -0.013619 -0.056081  0.035009  0.084344  0.072522   
12324  0.074079  0.017636 -0.037147  0.025048  0.063006 -0.024143 -0.002338   

          dim_7     dim_8     dim_9  ...   dim_376   dim_377   dim_378  \
3016  -0.010634 -0.062965  0.008658  ... -0.009284 -0.032884 -0.056897   
10784 -0.097514 -0.07503

In [8]:
def df_between_dates(df, start_date, end_date):
    return df[(df['date'] >= start_date) & (df['date'] <= end_date)]

now = datetime.now()
past_year = now + relativedelta(years=-1)
#df_week = df_between_dates(headline_df, '2024-11-01', '2024-12-3')
df = df_between_dates(headline_df, past_year.strftime("%Y-%m-%d"), now.strftime("%Y-%m-%d"))

In [9]:
emb_model = SentenceTransformer('all-MiniLM-L6-v2')

2024-12-30 10:20:42,792 INFO: Use pytorch device_name: cpu
2024-12-30 10:20:42,794 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [10]:
headlines= df['headlines'].tolist()
embeddings = df[[f"dim_{i}" for i in range(384)]].to_numpy()#emb_model.encode(headlines[key])

In [12]:
def training(embeddings, headlines, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=22)
    clusters = kmeans.fit(embeddings) # 1, 2 or 3... for each headline
    pca = PCA(n_components=2)
    pca.fit(embeddings)
    return kmeans, pca
n_clusters=5
kmeans,pca = training(embeddings, headlines, n_clusters)

In [13]:
emb =  emb_model.encode(['police', 'Trump'])
clusters = kmeans.predict(emb)
print(clusters)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[2 4]


In [14]:
import pickle
with open("pca_model.pkl", "wb") as pca_file:
    pickle.dump(pca, pca_file)

with open("kmeans_model.pkl", "wb") as kmeans_file:
    pickle.dump(kmeans, kmeans_file)

In [15]:
mr = project.get_model_registry()

pca_model = mr.python.create_model(
    name="PCA_Model",
    description="PCA model for dimensionality reduction"
)
kmeans_model = mr.python.create_model(
    name="KMeans_Model",
    description="KMeans clustering model"
)
pca_model.save('pca_model.pkl')
kmeans_model.save('kmeans_model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/5399 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1163414/models/PCA_Model/12


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/62027 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1163414/models/KMeans_Model/12


Model(name: 'KMeans_Model', version: 12)

Look at size

In [16]:
print(embeddings.shape)
print(len(headlines))

(13424, 384)
13424


In [17]:
#size mb, headlines 1000 times larger
import sys
print(sys.getsizeof(embeddings)/ (1024**2))
print(sys.getsizeof(headlines)/ (1024**2))

0.0001220703125
0.10247039794921875
